In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score

In [94]:
df = pd.read_csv('datasets/horse.csv')
df.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

In [96]:
df.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [97]:
cols = df.columns

In [98]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(df)
df = imputer.transform(df)

In [99]:
df = pd.DataFrame(df, columns=cols)

In [100]:
df = df.apply(pd.to_numeric, errors='ignore')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            299 non-null    float64
 4   pulse                  299 non-null    float64
 5   respiratory_rate       299 non-null    float64
 6   temp_of_extremities    299 non-null    object 
 7   peripheral_pulse       299 non-null    object 
 8   mucous_membrane        299 non-null    object 
 9   capillary_refill_time  299 non-null    object 
 10  pain                   299 non-null    object 
 11  peristalsis            299 non-null    object 
 12  abdominal_distention   299 non-null    object 
 13  nasogastric_tube       299 non-null    object 
 14  nasogastric_reflux     299 non-null    object 
 15  nasoga

C:\Users\praka\AppData\Local\Temp\ipykernel_10480\2930620201.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [101]:
df.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,normal_pink,more_3_sec,...,45.0,8.4,cloudy,2.0,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,cool,normal,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,cloudy,2.0,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,cool,normal,dark_cyanotic,more_3_sec,...,74.0,7.4,cloudy,2.0,died,no,4300,0,0,no


In [102]:
df.isnull().sum()

surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
outcome                  0
surgical_lesion          0
lesion_1                 0
lesion_2                 0
lesion_3                 0
cp_data                  0
dtype: int64

In [103]:
le = LabelEncoder()
df['cp_data'] = le.fit_transform(df['cp_data'])
df['cp_data'].unique()

array([0, 1])

In [104]:
object_columns = df.select_dtypes(include=['object']).columns
print(object_columns)

Index(['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'outcome',
       'surgical_lesion'],
      dtype='object')


In [105]:
object_columns.to_list()

['surgery',
 'age',
 'temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'outcome',
 'surgical_lesion']

In [106]:
y = df['cp_data'].values

df.drop(['cp_data'], axis = 1, inplace=True)

df.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,normal_pink,more_3_sec,...,distend_large,45.0,8.4,cloudy,2.0,died,no,11300,0,0
1,yes,adult,534817,39.2,88.0,20.0,cool,normal,pale_cyanotic,less_3_sec,...,other,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,normal,33.0,6.7,cloudy,2.0,lived,no,0,0,0
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,distend_large,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0
4,no,adult,530255,37.3,104.0,35.0,cool,normal,dark_cyanotic,more_3_sec,...,distend_large,74.0,7.4,cloudy,2.0,died,no,4300,0,0


In [107]:
ct = ColumnTransformer(transformers=[
     ('Encoder', OneHotEncoder(drop='first', sparse_output=False), object_columns.to_list())
    ], remainder='passthrough')

X = ct.fit_transform(df)

In [108]:
ct.get_feature_names_out()

array(['Encoder__surgery_yes', 'Encoder__age_young',
       'Encoder__temp_of_extremities_cool',
       'Encoder__temp_of_extremities_normal',
       'Encoder__temp_of_extremities_warm',
       'Encoder__peripheral_pulse_increased',
       'Encoder__peripheral_pulse_normal',
       'Encoder__peripheral_pulse_reduced',
       'Encoder__mucous_membrane_bright_red',
       'Encoder__mucous_membrane_dark_cyanotic',
       'Encoder__mucous_membrane_normal_pink',
       'Encoder__mucous_membrane_pale_cyanotic',
       'Encoder__mucous_membrane_pale_pink',
       'Encoder__capillary_refill_time_less_3_sec',
       'Encoder__capillary_refill_time_more_3_sec',
       'Encoder__pain_depressed', 'Encoder__pain_extreme_pain',
       'Encoder__pain_mild_pain', 'Encoder__pain_severe_pain',
       'Encoder__peristalsis_hypermotile',
       'Encoder__peristalsis_hypomotile', 'Encoder__peristalsis_normal',
       'Encoder__abdominal_distention_none',
       'Encoder__abdominal_distention_severe',
     

In [109]:
print(X, "\n\n", X.shape, "\n\n" , y, "\n\n", y.shape)

[[0.000e+00 0.000e+00 1.000e+00 ... 1.130e+04 0.000e+00 0.000e+00]
 [1.000e+00 0.000e+00 1.000e+00 ... 2.208e+03 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [1.000e+00 0.000e+00 0.000e+00 ... 3.205e+03 0.000e+00 0.000e+00]
 [1.000e+00 0.000e+00 1.000e+00 ... 2.208e+03 0.000e+00 0.000e+00]
 [1.000e+00 0.000e+00 1.000e+00 ... 6.112e+03 0.000e+00 0.000e+00]] 

 (299, 52) 

 [0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1
 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 1
 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0
 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1
 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1
 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 

Decision Tree - 80% Accuracy

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

dtree_classifier = DecisionTreeClassifier(random_state = 0)
dtree_classifier.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [111]:
y_pred = dtree_classifier.predict(X_test)

In [112]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[29  6]
 [ 6 19]]


0.8

Random Forest - 78.3% Accuracy

In [113]:
rf_classifier = RandomForestClassifier(random_state = 0, n_estimators=50)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=0)

In [114]:
y_pred = rf_classifier.predict(X_test)

In [115]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[32  3]
 [10 15]]


0.7833333333333333